# 실험 04-01 제너 다이오드 응용 회로

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    elif (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 시뮬레이션 1-1 결과 분석

## 회로도

* 사용한 컴포넌트
    + voltage (V): V1
    + res (R): R1
    + zener diode (Component -> zenor) : D1
    + ground (G): Ground
    + Net Name (F4): VS, VO
* 시뮬레이션 설정
    + SPICE directive (S)
        - .tran 0 500m 0 10u
        - .step param VDC 1V 14V 1V
* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)

    ![Markdown-mark](../ltspice/simulation_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01.txt')

### 2. 시뮬레이션 결과에서 시간, $V_Z$((VO)), $I_Z$(I(D1))를 가져온다.

In [ ]:
times = []
vzs = []
izs = []
vss = []

for idx in range(1, 15):
    vss.append(idx*1000)

for idx in range(1, len(vss)+1):
    times.append(data['(TIME)@%d' % idx])
    vzs.append(data['(V(VO))@%d' % idx])
    izs.append(-data['(I(D1))@%d' % idx])


### 3. 시간을 x축, $V_Z$와 $I_Z$를 각각 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

for idx in range(len(vss)):
    draw_plot(times[idx], vzs[idx], 'V_Z (V_S=%dV)' % (vss[idx]/1000), idx, marker_num=1)

plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

for idx in range(len(vss)):
    draw_plot(times[idx], izs[idx], 'I_Z (V_S=%dV)' % (vss[idx]/1000), idx, marker_num=1)

plt.show()

### 4. $V_Z$를 x축, $I_Z$를 y축으로 하는 그래프를 그린다.

In [ ]:
iz = []
vz = []

for idx in range(len(vss)):
    iz.append(izs[idx][25])
    vz.append(vzs[idx][25])

iz = np.array(iz)
vz = np.array(vz)

print_array('V_S_mv', vss)
print_array('V_Z_mV', vz)
print_array('I_Z_mA', iz)


In [ ]:
plt.ylabel('I_Z_mA')
plt.xlabel('V_Z_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

draw_plot(vz, iz, None, 0, marker_num=-1)

plt.show()

In [ ]:
plt.ylabel('- I_Z_mA')
plt.xlabel('- V_Z_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

draw_plot(-vz, -iz, None, 0, marker_num=-1)

ax = plt.gca()
ax.set_xlim([-6000, 6000])
ax.set_ylim([-90, 90])
plt.show()

*******************************************************************************
## 시뮬레이션 1-2 결과 분석

## 회로도

* 사용한 컴포넌트
    + voltage (V): V1
    + res (R): R1, R2
    + zener diode (Component -> zenor) : D1
    + ground (G): Ground
    + Net Name (F4): VS, VO
* 시뮬레이션 설정
    + SPICE directive (S)
        - .tran 0 500m 0 10u
        - ..step param RL list 0.047k, 0.1k, 0.47k, 1.0k, 3.3k
* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)

    ![Markdown-mark](../ltspice/simulation_02-01.png)
    ![Markdown-mark](../ltspice/simulation_02-02.png)


### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02-01.txt')

In [ ]:
data2 = get_simulation_result('../ltspice/simulation_02-02.txt')

### 2. 시뮬레이션 결과에서 시간, $V_O$((VO)), $I_Z$(I(D1)), $I_L$(I(R2))를 가져온다.

In [ ]:
times = []
vos = []
izs = []
ils = []

rls = np.array([47., 100., 470., 1000., 3300.])

for idx in range(1, len(rls)+1):
    times.append(data['(TIME)@%d' % idx])
    vos.append(data['(V(VO))@%d' % idx])
    izs.append(-data['(I(D1))@%d' % idx])
    ils.append(data['(I(R2))@%d' % idx])

times2 = data2['TIME']
vos2 = data2['V(VO)']
izs2 = -data2['I(D1)']

### 3. 시간을 x축, $V_Z$, $I_Z$, $I_ㅣ$를 각각 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

for idx in range(len(rls)):
    draw_plot(times[idx], vos[idx], 'V_O (R_L=%0.3fkOhm)' % (rls[idx]/1000), idx, marker_num=1)

draw_plot(times2, vos2, 'V_O (R_L=inf.)', len(rls), marker_num=1)

plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

for idx in range(len(rls)):
    draw_plot(times[idx], izs[idx], 'I_Z (R_L=%0.3fkOhm)' % (rls[idx]/1000), idx, marker_num=1)

draw_plot(times2, izs2, 'I_Z (R_L=inf.)', len(rls), marker_num=1)

plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

for idx in range(len(rls)):
    draw_plot(times[idx], ils[idx], 'I_L (R_L=%0.3fkOhm)' % (rls[idx]/1000), idx, marker_num=1)

plt.show()

### 4. $R_L$을 x축, $V_O$, $I_Z$ 및 $I_L$를 각각 y축으로 하는 그래프를 그린다.

In [ ]:
iz = []
il = []
vo = []

for idx in range(len(rls)):
    vo.append(vos[idx][25])
    iz.append(izs[idx][25])
    il.append(ils[idx][25])

vo = np.array(vo)
iz = np.array(iz)
il = np.array(il)

vo2 = vos2[25]
iz2 = izs2[25]

print_array('R_L_Ohm', rls)
print_array('V_O_mV ', vo)
print_value('V_O_mV (R_L=inf.)', vo2)
print_array('I_Z_mA ', iz)
print_value('I_Z_mA (R_L=inf.)', iz2)
print_array('I_L_mA ', il)

In [ ]:
plt.xlabel('R_L_ohm')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

draw_plot(rls, vo, 'V_O', 0, marker_num=-1)

plt.show()

In [ ]:
plt.xlabel('R_L_ohm')
plt.ylabel('I_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')

draw_plot(rls, iz, 'I_Z', 1, marker_num=-1)
draw_plot(rls, il, 'I_L', 2, marker_num=-1)

plt.show()